In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd

def read_zhuyin2upa_table():
    zdf = pd.read_excel("zhuyin2ipa.xlsx")
    return zdf

In [3]:
SPECZ1=['ㄓ','ㄔ','ㄕ','ㄖ','ㄗ','ㄘ','ㄙ']
SPECZ2=['ㄧ','ㄨ','ㄩ']
FOURSOUNDS=['ˊ','ˇ','ˋ','˙']
DUOZ=['ㄧㄢ','ㄩㄢ','ㄧㄣ','ㄧㄥ','ㄨㄥ','ㄩㄥ']

def is_ascii(check_str):
     return all(ord(c) < 128 for c in check_str)

def foursounds(c):
    if 'ˋ' in c:
        return "51 "
    elif 'ˊ' in c:
        return "35 "
    elif 'ˇ' in c:
        return "21 "
    elif '˙' in c:
        return "0 "
    else:
        return "55 "
    
def convertZH(zdf, zh):
    #print("searching", zh)
    x = zdf[zdf['Zhuyin']==zh].IPA.to_string(index=False)
    if x == "Series([], )":
        return ''
    else:
        return x.replace(' ','')

def remove_foursounds(zh):
    z = zh
    for f in FOURSOUNDS:
        z = z.replace(f,'')
    return z

def is_duo(zh):
    for d in DUOZ:
        if d in zh:
            return True
    return False

def duo_convert(zdf, zh):
    #print("zh0",zh[0])
    matching = False
    for z in SPECZ2:
        #print("zzh", z, zh[0])
        if z == zh[0]:
            matching = True
            break
    
    if matching:
        #print("Converting zh", zh)
        return convertZH(zdf, zh)
    else:
        return convertZH(zdf, zh[0])+convertZH(zdf, zh[1:]).replace('\n','')
    
def zhuyin2ipa(zdf, zhuyin):
    zh=remove_foursounds(zhuyin)
    idx = 0
    zhlen = len(zh)
    
    if is_duo(zh):
        return(duo_convert(zdf, zh))
    
    IPAL=[]
    while (idx < zhlen):
        #print(idx)
        if zh[idx] in SPECZ1:
            if zhlen == 1:
                return (convertZH(zdf, zh[idx])+'ɨ')
            else:
                IPAL.append(convertZH(zdf, zh[idx]))
                idx = idx + 1
                
            if zh[idx] in SPECZ2:
                if idx == zhlen - 1:
                    #The last one
                    IPAL.append(convertZH(zdf, zh[idx]))
                    return IPAL
        else:
            IPAL.append(convertZH(zdf, zh[idx]))
            idx = idx + 1
            
    #print(IPAL)
    return IPAL


In [4]:
def chine2ipa(df, zdf):
    col = df.columns.values
    row = df.index.values
    newdf = df.copy()

    for c in col:
        for r in row:
            ipastr=''
            x=str(df.loc[r,c]).replace(' ','')
            xx = pinyin(x, style=Style.BOPOMOFO)
            for item in xx:
                if not is_ascii(item[0]):
                    ipal=zhuyin2ipa(zdf, item[0])
                    fstr=foursounds(item[0])
                    for iipa in ipal:
                        ipastr = ipastr + iipa
                    ipastr = ipastr + foursounds(item[0])
                else:
                    ipastr = ipastr+item[0]
            newdf.loc[r,c] = str(df.loc[r,c])+' '+ipastr
    return newdf

In [5]:
def replace_oo_with_ipa():
    return

In [6]:
def contains_oo(x):
    if 'o|o' in x:
        return True
    else:
        return False
    
def contains_oneo(x):
    if 'o|' in x or '|o' in x:
        return True
    else:
        return False

def replace_o(x, col):
    return

In [69]:
def excel_page1(zdf):
    # to read just one sheet to dataframe:
    df = pd.read_excel('exp123.xlsx', sheet_name="實驗一", index_col=0, head=0)
    col = df.columns.values
    row = df.index.values

    for c in col:
        for r in row:
            x=str(df.loc[r,c]).lstrip().strip().replace(' ','|').replace('oo','o|o').replace('xx','x|x').replace('??','?|?').replace('ox','o|x').replace('xo','x|o').split('-')
            #print(x)
            if contains_oo(x) or contains_oneo(x):
                ttlist = r.split()
                cl = []
                for t in ttlist:
                    term = list(t)
                    cl.append(term[0])
                    cl.append(term[1])
                
                xl = []
                for xi in x:
                    xt = xi.split('|')
                    xl.append(xt[0])
                    if(len(xt) == 2):
                        xl.append(xt[1])
                    
                print(cl,xl)
            #    y = replace_o(x, cl)
            xx=[b.count('|') for b in x if b.count('|') == 1]
            if (sum(xx) != 4):
                print(c,r, x)
    #newdf = df.copy()
    #
    #newdf.to_excel('實驗ㄧ.xlsx')
#excel_page1()

In [70]:
def excel_page2(zdf):
    # to read just one sheet to dataframe:
    df = pd.read_excel('exp123.xlsx', sheet_name="實驗二", index_col=0, head=0)
    col = df.columns.values
    row = df.index.values

    print(row)

    for c in col:
        for r in row:
            x=str(df.loc[r,c]).lstrip().strip().replace(' ','|').replace('oo','o|o').replace('xx','x|x').replace('??','?|?').replace('ox','o|x').replace('xo','x|o').split('-')
            #print(x)
            xx=[b.count('|') for b in x if b.count('|') == 1]
            if (sum(xx) != 4):
                print(c,r, x)
    #newdf = df.copy()
    #
    #newdf.to_excel('實驗ㄧ.xlsx')

#excel_page2()

In [71]:
from pypinyin import pinyin, lazy_pinyin, Style

def excel_page3(zdf):
    df = pd.read_excel('exp123.xlsx', sheet_name="實驗三(女)",index_col=0, head=0)
    newdf = chine2ipa(df, zdf)
    newdf.to_excel('實驗三(女).xlsx')

#excel_page3(zdf)

In [72]:
# to read just one sheet to dataframe:

def excel_page4(zdf):
    df = pd.read_excel('exp123.xlsx', sheet_name="實驗三(男)",index_col=0, head=0)
    newdf = chine2ipa(df, zdf)
    newdf.to_excel('實驗三(男).xlsx')

In [73]:
def main():
    zdf = read_zhuyin2upa_table()
    excel_page1(zdf)
    excel_page2(zdf)
    excel_page3(zdf)
    excel_page4(zdf)
    
if __name__ == '__main__':
    main()  # 或是任何你想執行的函式


['大', '菟', '踏', '渡', '但', '套', '嘆', '稻'] ['o', 'o', 'tɑw51', 'tʰu51', 'o', 'o', 'x', 'x']
['岔', '怒', '吶', '絀', '顛', '鬧', '難', '潮'] ['o', 'o', 'o', 'o', 'x', 'x', 'x', 'x']
['壩', '牧', '罵', '怖', '絆', '貿', '蔓', '爆'] ['o', 'o', 'pɑw51', 'pu51', 'pa51', 'nu51', 'x', 'x']
['綻', '怒', '納', '助', '湛', '鬧', '難', '兆'] ['(大怒)o', 'o', 'x', 'x', 'o', 'o', 'x', 'x']
['他', '租', '紮', '禿', '灘', '遭', '簪', '掏'] ['o', 'o', 'ta51', 'nu51', 'pɑw51', 'nu51', 'x', 'x']
['爸', '度', '大', '怖', '半', '到', '但', '抱'] ['o', 'o', 'ta51', 'nu51', 'pu51', 'tu51', 'x', 'x']
['蹋', '注', '詐', '吐', '探', '趙', '棧', '套'] ['o', 'tu51', 'o', 'o', 'ta51', 'x', 'x', 'x']
['颯', '怒', '納', '粟', '散', '鬧', '難', '臊'] ['o', 'o', 'o', 'o', 'x', 'x', 'x', 'x']
['榨', '陸', '臘', '祝', '綻', '烙', '爛', '趙'] ['x', 'x', 'o', 'o', 'o', 'nu51', 'tʂan51', 'x']
['塌', '書', '沙', '禿', '攤', '稍', '山', '滔'] ['o', 'o', 'o', 'o', 'x', 'x', 'x', 'x']
F1 颯穆 罵宿 散冒 曼臊 ['o|nu51', 'o|su51', 'sa51|mɑw51']
['罷', '瀑', '怕', '不', '辦', '泡', '盼', '報'] ['pa51', 'o', 'o', 'o', '